In [1]:
from xml.etree import ElementTree
import pandas as pd
import re

In [2]:
def format_question(q):
    del q.attrib['Body']
    q.attrib['Score'] = int(q.attrib['Score'])
    tags = re.findall('\[(.*?)\]', q.attrib['Title'])
    return {**q.attrib, 'tags_in_title': tags or None}

In [3]:
df = pd.DataFrame(
    format_question(elem)
    for _, elem in ElementTree.iterparse('../downloads/MSO/Posts.xml')
    if elem.attrib.get('PostTypeId') == '1'
)

In [4]:
requests = df.Tags.str.extractall('(burninate|synonym|retag)-request>')
status = df.Tags.str.extract('<(status-.*?)>', expand=False).fillna('*NO-STATUS*').rename('status')

In [5]:
for_review = (
    df[['Id', 'CreationDate', 'Title', 'Score', 'ViewCount', 'tags_in_title']]
    .join([pd.get_dummies(requests, prefix='tagged').groupby(level=0).max(), status], how='inner')
    .sort_values('Score', ascending=False)
)

In [6]:
for_review.to_html('draft_review.html', index=False)

In [7]:
for_review

,Id,CreationDate,Title,Score,ViewCount,tags_in_title,tagged_burninate,tagged_retag,tagged_synonym,status
23451,331817,2016-08-06T20:05:57.070,Should we burninate [godaddy]?,390,16344,[godaddy],1,0,0,status-completed
24170,333833,2016-09-04T15:32:05.667,Should we burninate the [Apple] tag?,295,8576,[Apple],1,0,0,status-completed
1527,255169,2014-05-16T16:29:56.670,I want to delete the [internet],192,3796,[internet],1,0,0,status-completed
13259,298124,2015-06-29T15:20:36.740,Stop the [manipulation]!,189,3054,[manipulation],1,0,0,status-completed
20472,321966,2016-04-28T07:38:32.173,This tag has been taken off [track],185,3318,[track],1,0,0,status-completed
20860,323081,2016-05-15T19:59:18.413,We have stopped doing [research],177,3212,[research],1,1,0,status-completed
31018,355395,2017-08-21T02:21:37.203,The [convert] tag has been burninated,164,3822,[convert],1,0,0,status-completed
29474,349778,2017-05-26T21:30:34.880,Another [entry] on the burnination list,161,3275,[entry],1,0,0,status-completed
15956,308792,2015-10-26T08:32:00.713,Let's clean up [friendship],153,9849,[friendship],1,0,0,status-completed
2283,258571,2014-06-06T01:19:37.200,Should we burninate this tag [once] and for all?,146,4351,[once],1,0,0,status-completed
